In [1]:
# !pip install -U kaleido

/bin/bash: line 1: $: command not found


In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.mode.chained_assignment = None

In [10]:
# carregar dados
df_sgtf_original = pd.read_csv('F:\Dados GISAID\dengue\dengueBR_simples_20240416.tsv\dengueBR_simples.tsv',
                               sep='\t')
df_sgtf_original.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO
0,2019-01-04,AC,NaN
1,2019-04-10,AC,NaN
2,2019-12-14,AC,NaN
3,2019-11-19,AC,NaN
4,2019-09-23,AC,NaN


In [11]:
# cópia para df só do BR
df_sgtf_br = df_sgtf_original.copy()

## Brasil

In [12]:
# criar coluna Período (Mes e Ano)
df_sgtf_br['Período'] = df_sgtf_br['DT_NOTIFIC'].str[:7]
df_sgtf_br.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO,Período
0,2019-01-04,AC,NaN,2019-01
1,2019-04-10,AC,NaN,2019-04
2,2019-12-14,AC,NaN,2019-12
3,2019-11-19,AC,NaN,2019-11
4,2019-09-23,AC,NaN,2019-09


In [13]:
df_sgtf_br = df_sgtf_br.drop('DT_NOTIFIC', axis=1)

# Substituir os NAN
df_sgtf_br['SOROTIPO'] = df_sgtf_br['SOROTIPO'].fillna('Sem informação')
df_sgtf_br['SG_UF_NOT'] = df_sgtf_br['SG_UF_NOT'].fillna('Sem informação')

In [14]:
# agrupar por período e sorotipo
df_sgtf_br_agrupado = df_sgtf_br.groupby(['Período', 'SOROTIPO']).count().reset_index()
df_sgtf_br_agrupado = df_sgtf_br_agrupado.sort_values('Período').reset_index(drop=True)
df_sgtf_br_agrupado.head(2)

,Período,SOROTIPO,SG_UF_NOT
0,2007-01,1,70
1,2007-01,2,19


In [15]:
# criar coluna ANO (fixar junho)
df_sgtf_br_agrupado['Ano'] = df_sgtf_br_agrupado['Período'].str[:4] + '-06'
df_sgtf_br_agrupado.head(2)

,Período,SOROTIPO,SG_UF_NOT,Ano
0,2007-01,1,70,2007-06
1,2007-01,2,19,2007-06


In [16]:
# renomear colunas
df_sgtf_br_agrupado = df_sgtf_br_agrupado.rename(columns={'SG_UF_NOT': 'Casos',
                                                          'SOROTIPO': 'Sorotipo'})
df_sgtf_br_agrupado.head(1)

,Período,Sorotipo,Casos,Ano
0,2007-01,1,70,2007-06


In [17]:
# total de casos
df_sgtf_br_agrupado['Casos'].sum()

24031903

### ANO

In [19]:
# criar um df agrupado apenas pelo ANO para as linhas de frequência anuais de cada sorotipo
df_sgtf_br_agrupado_ano = df_sgtf_br_agrupado[['Ano', 'Sorotipo', 'Casos']].groupby(['Ano', 'Sorotipo']).sum()
df_sgtf_br_agrupado_ano = df_sgtf_br_agrupado_ano.reset_index()
df_sgtf_br_agrupado_ano

,Ano,Sorotipo,Casos
0,2007-06,1,1797
1,2007-06,2,358
2,2007-06,3,951
3,2007-06,4,215
4,2007-06,Sem informação,713776
...,...,...,...
80,2024-06,1,39471
81,2024-06,2,19074
82,2024-06,3,300
83,2024-06,4,25


In [20]:
# criar em DF com a porcentagem de sorotipo por caso por ANO
# lista com os ANOS
lista_anos = df_sgtf_br_agrupado_ano['Ano'].unique()
# df vazio
df_sgtf_br_agrupado_ano_freq = pd.DataFrame(columns=['Ano', 'Sorotipo', 'Casos'])

# percorrer cada período
for ano in lista_anos:
    # filtrar o ANO
    df_filtrado_total = df_sgtf_br_agrupado_ano.query('Ano == @ano')
    # filtrar o ANO e sorotipo
    df_filtrado_sorotipo = df_sgtf_br_agrupado_ano.query('Ano == @ano and Sorotipo != "Sem informação"')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo['Casos'].sum()

    # porcentagem de cada sorotipo por ANO pelo total de casos do ANO
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_sgtf_br_agrupado_ano_freq = pd.concat([df_sgtf_br_agrupado_ano_freq, df_filtrado_sorotipo])

In [21]:
df_sgtf_br_agrupado_ano_freq['Sorotipo'] = df_sgtf_br_agrupado_ano_freq['Sorotipo'].astype('str').str[0]

df_sorotipo1_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "1"').reset_index(drop=True)
df_sorotipo2_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "2"').reset_index(drop=True)
df_sorotipo3_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "3"').reset_index(drop=True)
df_sorotipo4_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "4"').reset_index(drop=True)

### Período

In [26]:
# criar em DF com a porcentagem de sorotipo por caso por período
# lista com os períodos
lista_periodos = df_sgtf_br_agrupado['Período'].unique()
# df vazio
df_sgtf_br_agrupado_periodo_freq = pd.DataFrame(columns=['Período', 'Sorotipo', 'Casos'])

# percorrer cada período
for periodo in lista_periodos:
    # filtrar o período
    df_filtrado_total = df_sgtf_br_agrupado.query('Período == @periodo')
    # filtrar o período e sorotipo
    df_filtrado_sorotipo = df_sgtf_br_agrupado.query('Período == @periodo and Sorotipo != "Sem informação"')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo['Casos'].sum()

    # porcentagem de cada sorotipo por período pelo total de casos do período
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_sgtf_br_agrupado_periodo_freq = pd.concat([df_sgtf_br_agrupado_periodo_freq, df_filtrado_sorotipo])
    print(df_sgtf_br_agrupado_periodo_freq.head(2))

   Período Sorotipo Casos      Ano  CasosExtrap  Porcentagem
0  2007-01        1    70  2007-06      17000.0        34.31
1  2007-01        2    19  2007-06       4614.0         9.31
   Período Sorotipo Casos      Ano  CasosExtrap  Porcentagem
0  2007-01        1    70  2007-06      17000.0        34.31
1  2007-01        2    19  2007-06       4614.0         9.31
   Período Sorotipo Casos      Ano  CasosExtrap  Porcentagem
0  2007-01        1    70  2007-06      17000.0        34.31
1  2007-01        2    19  2007-06       4614.0         9.31
   Período Sorotipo Casos      Ano  CasosExtrap  Porcentagem
0  2007-01        1    70  2007-06      17000.0        34.31
1  2007-01        2    19  2007-06       4614.0         9.31
   Período Sorotipo Casos      Ano  CasosExtrap  Porcentagem
0  2007-01        1    70  2007-06      17000.0        34.31
1  2007-01        2    19  2007-06       4614.0         9.31
   Período Sorotipo Casos      Ano  CasosExtrap  Porcentagem
0  2007-01        1    7

In [27]:
df_sgtf_br_agrupado_periodo_freq['Sorotipo'] = df_sgtf_br_agrupado_periodo_freq['Sorotipo'].astype('str')
df_sgtf_br_agrupado_periodo_freq['Sorotipo'] = df_sgtf_br_agrupado_periodo_freq['Sorotipo'].str[0]

df_sorotipo1_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "1"').reset_index(drop=True)
df_sorotipo2_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "2"').reset_index(drop=True)
df_sorotipo3_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "3"').reset_index(drop=True)
df_sorotipo4_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "4"').reset_index(drop=True)

In [28]:
# exportar dfs
df_sgtf_br_agrupado_ano_freq.to_csv('df_dengue_pcr_ano.csv', index=False)
df_sgtf_br_agrupado_periodo_freq.to_csv('df_dengue_pcr_periodo.csv', index=False)

### Gráfico

In [29]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Sorotipo 1',
                     marker_color='mediumvioletred',
                     x=df_sorotipo1_periodo['Período'],
                     y=df_sorotipo1_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 2',
                     marker_color='darkblue',
                     x=df_sorotipo2_periodo['Período'],
                     y=df_sorotipo2_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 3',
                     marker_color='green',
                     x=df_sorotipo3_periodo['Período'],
                     y=df_sorotipo3_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 4',
                     marker_color='gold',
                     x=df_sorotipo4_periodo['Período'],
                     y=df_sorotipo4_periodo['CasosExtrap']),
              secondary_y=False)

# LINHA
fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='mediumvioletred',
                         x=df_sorotipo1_ano['Ano'],
                         y=df_sorotipo1_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='darkblue',
                         x=df_sorotipo2_ano['Ano'],
                         y=df_sorotipo2_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='green',
                         x=df_sorotipo3_ano['Ano'],
                         y=df_sorotipo3_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='gold',
                         x=df_sorotipo4_ano['Ano'],
                         y=df_sorotipo4_ano['Porcentagem']),
              secondary_y=True)

fig.update_layout(barmode='stack',
                  legend_font_size=14,
                  legend_traceorder='normal',
                  legend_orientation='h',
                  legend_xanchor = 'center',
                  legend_yanchor = 'bottom',
                  legend_x= 0.5, legend_y= 1.0,
                  plot_bgcolor='white',
                  xaxis_title="Período de amostragem",
                  yaxis_title="Casos de dengue",)

fig.update_xaxes(showline=True, linewidth=1.5, linecolor='lightgray')
fig.update_yaxes(showline=True, linewidth=1.5, linecolor='lightgray', secondary_y=False)
fig.update_yaxes(secondary_y=True, showline=True, linewidth=1.5, linecolor='lightgray',
                 title_text='Sorotipos (%)', rangemode="tozero")

fig.show()

## Regiões

In [30]:
# cópia para df só de região
df_sgtf_regiao = df_sgtf_original.copy()

In [31]:
# criar coluna Período (Mes e Ano)
df_sgtf_regiao['Período'] = df_sgtf_regiao['DT_NOTIFIC'].str[:7]
df_sgtf_regiao.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO,Período
0,2019-01-04,AC,NaN,2019-01
1,2019-04-10,AC,NaN,2019-04
2,2019-12-14,AC,NaN,2019-12
3,2019-11-19,AC,NaN,2019-11
4,2019-09-23,AC,NaN,2019-09


In [32]:
df_sgtf_regiao = df_sgtf_regiao.drop('DT_NOTIFIC', axis=1)

# Substituir os NAN
df_sgtf_regiao['SOROTIPO'] = df_sgtf_regiao['SOROTIPO'].fillna('Sem informação')
df_sgtf_regiao['SG_UF_NOT'] = df_sgtf_regiao['SG_UF_NOT'].fillna('Sem informação')

In [33]:
# criar Região
dict_regiao = {"AC": "Norte",
               "AM": "Norte",
               "RR": "Norte",
               "RO": "Norte",
               "PA": "Norte",
               "AP": "Norte",
               "TO": "Norte",
               "MA": "Nordeste",
               "PI": "Nordeste",
               "CE": "Nordeste",
               "RN": "Nordeste",
               "PB": "Nordeste",
               "PE": "Nordeste",
               "AL": "Nordeste",
               "SE": "Nordeste",
               "BA": "Nordeste",
               "MT": "Centro-oeste",
               "MS": "Centro-oeste",
               "GO": "Centro-oeste",
               "DF": "Centro-oeste",
               "SP": "Sudeste",
               "RJ": "Sudeste",
               "MG": "Sudeste",
               "ES": "Sudeste",
               "PR": "Sul",
               "SC": "Sul",
               "RS": "Sul",
               "Sem informação": "Sem informação"}

In [34]:
df_sgtf_regiao['Região'] = df_sgtf_regiao['SG_UF_NOT'].map(dict_regiao)
df_sgtf_regiao.head(1)

,SG_UF_NOT,SOROTIPO,Período,Região
0,AC,Sem informação,2019-01,Norte


In [35]:
df_sgtf_regiao['Região'].value_counts()

Sudeste           12458170
Nordeste           4073041
Centro-oeste       3661330
Sul                2566460
Norte              1272899
Sem informação           3
Name: Região, dtype: int64

In [36]:
# agrupar por período, região e sorotipo
df_sgtf_regiao_agrupado = df_sgtf_regiao.groupby(['Região', 'Período', 'SOROTIPO']).count().reset_index()
df_sgtf_regiao_agrupado = df_sgtf_regiao_agrupado.sort_values('Período').reset_index(drop=True)
df_sgtf_regiao_agrupado.head(2)

,Região,Período,SOROTIPO,SG_UF_NOT
0,Centro-oeste,2007-01,1,17
1,Sul,2007-01,3,6


In [37]:
# criar coluna ANO (fixar junho)
df_sgtf_regiao_agrupado['Ano'] = df_sgtf_regiao_agrupado['Período'].str[:4] + '-06'
df_sgtf_regiao_agrupado.head(2)

,Região,Período,SOROTIPO,SG_UF_NOT,Ano
0,Centro-oeste,2007-01,1,17,2007-06
1,Sul,2007-01,3,6,2007-06


In [38]:
# renomear colunas
df_sgtf_regiao_agrupado = df_sgtf_regiao_agrupado.rename(columns={'SG_UF_NOT': 'Casos',
                                                                  'SOROTIPO': 'Sorotipo'})
df_sgtf_regiao_agrupado.head(1)

,Região,Período,Sorotipo,Casos,Ano
0,Centro-oeste,2007-01,1,17,2007-06


In [39]:
# total de casos 22901570
df_sgtf_regiao_agrupado['Casos'].sum()

24031903

### ANO

In [40]:
# criar um df agrupado apenas pelo ANO para as linhas de frequência anuais de cada sorotipo
df_sgtf_regiao_agrupado_ano = df_sgtf_regiao_agrupado[['Região', 'Ano', 'Sorotipo', 'Casos']].groupby(['Região', 'Ano', 'Sorotipo']).sum()
df_sgtf_regiao_agrupado_ano = df_sgtf_regiao_agrupado_ano.reset_index()
df_sgtf_regiao_agrupado_ano

,Região,Ano,Sorotipo,Casos
0,Centro-oeste,2007-06,1,283
1,Centro-oeste,2007-06,2,5
2,Centro-oeste,2007-06,3,178
3,Centro-oeste,2007-06,4,30
4,Centro-oeste,2007-06,Sem informação,116953
...,...,...,...,...
392,Sul,2024-06,1,7384
393,Sul,2024-06,2,1576
394,Sul,2024-06,3,105
395,Sul,2024-06,4,5


In [41]:
# função para criar um DF com a porcentagem de sorotipo por caso por ANO e por Região
def calcula_frequencia_sorotipo_ano(df, regiao):
  df_filtrado = df.query('Região == @regiao')
  # lista com os ANOS
  lista_anos = df_filtrado['Ano'].unique()
  # cria df vazio
  df_final = pd.DataFrame(columns=['Região', 'Ano', 'Sorotipo', 'Casos'])

  # percorrer cada período
  for ano in lista_anos:
    # filtrar o ANO
    df_filtrado_total = df_filtrado.query('Ano == @ano')
    # filtrar o ANO e sorotipo
    df_filtrado_sorotipo = df_filtrado.query('Ano == @ano and Sorotipo != "Sem informação"')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo['Casos'].sum()

    # porcentagem de cada sorotipo por ANO pelo total de casos do ANO
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_final = pd.concat([df_final, df_filtrado_sorotipo])

  df_final['Sorotipo'] = df_final['Sorotipo'].astype('str').str[0]
  df_sorotipo1 = df_final.query('Sorotipo == "1"').reset_index(drop=True)
  df_sorotipo2 = df_final.query('Sorotipo == "2"').reset_index(drop=True)
  df_sorotipo3 = df_final.query('Sorotipo == "3"').reset_index(drop=True)
  df_sorotipo4 = df_final.query('Sorotipo == "4"').reset_index(drop=True)

  return df_sorotipo1, df_sorotipo2, df_sorotipo3, df_sorotipo4

### Período

In [42]:
# função para criar um DF com a porcentagem de sorotipo por caso por periodo e por Região
def calcula_frequencia_sorotipo_periodo(df, regiao):
  df_filtrado = df.query('Região == @regiao')
  # lista com os ANOS
  lista_periodos = df_filtrado['Período'].unique()
  # cria df vazio
  df_final = pd.DataFrame(columns=['Região', 'Período', 'Sorotipo', 'Casos'])

  # percorrer cada período
  for periodo in lista_periodos:
    # filtrar o periodo
    df_filtrado_total = df_filtrado.query('Período == @periodo')
    # filtrar o periodo e sorotipo
    df_filtrado_sorotipo = df_filtrado.query('Período == @periodo and Sorotipo != "Sem informação"')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo['Casos'].sum()

    # porcentagem de cada sorotipo por período pelo total de casos do período
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_final = pd.concat([df_final, df_filtrado_sorotipo])

  df_final['Sorotipo'] = df['Sorotipo'].astype('str').str[0]
  df_sorotipo1 = df_final.query('Sorotipo == "1"').reset_index(drop=True)
  df_sorotipo2 = df_final.query('Sorotipo == "2"').reset_index(drop=True)
  df_sorotipo3 = df_final.query('Sorotipo == "3"').reset_index(drop=True)
  df_sorotipo4 = df_final.query('Sorotipo == "4"').reset_index(drop=True)
  return df_sorotipo1, df_sorotipo2, df_sorotipo3, df_sorotipo4

In [44]:
# exportar dfs
df_sgtf_regiao_agrupado_ano.to_csv('df_dengue_pcr_regiao_ano.csv', index=False)
df_sgtf_regiao_agrupado.to_csv('df_dengue_pcr_regiao_periodo.csv', index=False)

### Gráfico

In [45]:
# ano
df_sudeste1_ano, df_sudeste2_ano, df_sudeste3_ano, df_sudeste4_ano = calcula_frequencia_sorotipo_ano(df_sgtf_regiao_agrupado_ano, 'Sudeste')
# periodo
df_sudeste1_per, df_sudeste2_per, df_sudeste3_per, df_sudeste4_per = calcula_frequencia_sorotipo_periodo(df_sgtf_regiao_agrupado, 'Sudeste')

In [46]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Sorotipo 1',
                     marker_color='mediumvioletred',
                     x=df_sudeste1_per['Período'],
                     y=df_sudeste1_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 2',
                     marker_color='darkblue',
                     x=df_sudeste2_per['Período'],
                     y=df_sudeste2_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 3',
                     marker_color='green',
                     x=df_sudeste3_per['Período'],
                     y=df_sudeste3_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 4',
                     marker_color='gold',
                     x=df_sudeste4_per['Período'],
                     y=df_sudeste4_per['CasosExtrap']),
              secondary_y=False)

# LINHA
fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='mediumvioletred',
                         x=df_sudeste1_ano['Ano'],
                         y=df_sudeste1_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='darkblue',
                         x=df_sudeste2_ano['Ano'],
                         y=df_sudeste2_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='green',
                         x=df_sudeste3_ano['Ano'],
                         y=df_sudeste3_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='gold',
                         x=df_sudeste4_ano['Ano'],
                         y=df_sudeste4_ano['Porcentagem']),
              secondary_y=True)

fig.update_layout(barmode='stack',
                  legend_font_size=14,
                  legend_traceorder='normal',
                  legend_orientation='h',
                  legend_xanchor = 'center',
                  legend_yanchor = 'bottom',
                  legend_x= 0.5, legend_y= 1.0,
                  plot_bgcolor='white',
                  xaxis_title="Período de amostragem",
                  yaxis_title="Casos de dengue",)

fig.update_xaxes(showline=True, linewidth=1.5, linecolor='lightgray')
fig.update_yaxes(showline=True, linewidth=1.5, linecolor='lightgray', secondary_y=False)
fig.update_yaxes(secondary_y=True, showline=True, linewidth=1.5, linecolor='lightgray',
                 title_text='Sorotipos (%)', rangemode="tozero")

fig.show()

## Estado

In [47]:
# cópia para df só de estado
df_sgtf_uf = df_sgtf_original.copy()

In [48]:
# criar coluna Período (Mes e Ano)
df_sgtf_uf['Período'] = df_sgtf_uf['DT_NOTIFIC'].str[:7]
df_sgtf_uf.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO,Período
0,2019-01-04,AC,NaN,2019-01
1,2019-04-10,AC,NaN,2019-04
2,2019-12-14,AC,NaN,2019-12
3,2019-11-19,AC,NaN,2019-11
4,2019-09-23,AC,NaN,2019-09


In [49]:
df_sgtf_uf = df_sgtf_uf.drop('DT_NOTIFIC', axis=1)

# Substituir os NAN
df_sgtf_uf['SOROTIPO'] = df_sgtf_uf['SOROTIPO'].fillna('Sem informação')
df_sgtf_uf['SG_UF_NOT'] = df_sgtf_uf['SG_UF_NOT'].fillna('Sem informação')

In [50]:
dict_uf = {'SP': 'São Paulo',
           'PB': 'Paraíba',
           'SE': 'Sergipe',
           'PI': 'Piauí',
           'PA': 'Pará',
           'MA': 'Maranhão',
           'AC': 'Acre',
           'AP': 'Amapá',
           'CE': 'Ceará',
           'ES': 'Espírito Santo',
           'RS': 'Rio Grande do Sul',
           'MG': 'Minas Gerais',
           'SC': 'Santa Catarina',
           'RR': 'Roraima',
           'AL': 'Alagoas',
           'PE': 'Pernambuco',
           'RN': 'Rio Grande do Norte',
           'MT': 'Mato Grosso',
           'PR': 'Paraná',
           'MS': 'Mato Grosso do Sul',
           'DF': 'Distrito Federal',
           'GO': 'Goiás',
           'BA': 'Bahia',
           'AM': 'Amazonas',
           'RO': 'Rondônia',
           'RJ': 'Rio de Janeiro',
           'TO': 'Tocantins'}

In [54]:
df_sgtf_uf['Estado'] = df_sgtf_uf['SG_UF_NOT'].map(dict_uf)
print(df_sgtf_uf.head(1))
print(df_sgtf_uf['SG_UF_NOT'].unique())


array(['AC', 'AM', 'RN', 'GO', 'DF', 'RJ', 'SP', 'PR', 'MS', 'AL', 'PE',
       'MG', 'MT', 'SE', 'ES', 'BA', 'CE', 'PB', 'RO', 'PA', 'RR', 'SC',
       'AP', 'RS', 'TO', 'PI', 'MA', 'Sem informação'], dtype=object)

In [55]:
df_sgtf_uf['Estado'].value_counts()

São Paulo              6124274
Minas Gerais           4409063
Paraná                 1882000
Goiás                  1841232
Rio de Janeiro         1412548
Bahia                  1063748
Ceará                   880219
Pernambuco              784227
Mato Grosso do Sul      722286
Distrito Federal        588540
Espírito Santo          512285
Mato Grosso             509272
Santa Catarina          454179
Rio Grande do Norte     386148
Alagoas                 299236
Acre                    275475
Tocantins               273353
Paraíba                 263670
Pará                    248124
Rio Grande do Sul       230281
Amazonas                219234
Piauí                   169232
Rondônia                161520
Maranhão                136723
Sergipe                  89838
Roraima                  52131
Amapá                    43062
Name: Estado, dtype: int64

In [56]:
# agrupar por período, uf e sorotipo
df_sgtf_uf_agrupado = df_sgtf_uf.groupby(['Estado', 'Período', 'SOROTIPO']).count().reset_index()
df_sgtf_uf_agrupado = df_sgtf_uf_agrupado.sort_values('Período').reset_index(drop=True)
df_sgtf_uf_agrupado.head(2)

,Estado,Período,SOROTIPO,SG_UF_NOT
0,Acre,2007-01,Sem informação,268
1,Distrito Federal,2007-01,Sem informação,166


In [57]:
# criar coluna ANO (fixar junho)
df_sgtf_uf_agrupado['Ano'] = df_sgtf_uf_agrupado['Período'].str[:4] + '-06'
df_sgtf_uf_agrupado.head(2)

,Estado,Período,SOROTIPO,SG_UF_NOT,Ano
0,Acre,2007-01,Sem informação,268,2007-06
1,Distrito Federal,2007-01,Sem informação,166,2007-06


In [58]:
# renomear colunas
df_sgtf_uf_agrupado = df_sgtf_uf_agrupado.rename(columns={'SG_UF_NOT': 'Casos',
                                                          'SOROTIPO': 'Sorotipo'})
df_sgtf_uf_agrupado.head(1)

,Estado,Período,Sorotipo,Casos,Ano
0,Acre,2007-01,Sem informação,268,2007-06


In [59]:
# total de casos 22901570
df_sgtf_uf_agrupado['Casos'].sum()

24031900

### ANO

In [60]:
# criar um df agrupado apenas pelo ANO para as linhas de frequência anuais de cada sorotipo
df_sgtf_uf_agrupado_ano = df_sgtf_uf_agrupado[['Estado', 'Ano', 'Sorotipo', 'Casos']].groupby(['Estado', 'Ano', 'Sorotipo']).sum()
df_sgtf_uf_agrupado_ano = df_sgtf_uf_agrupado_ano.reset_index()
df_sgtf_uf_agrupado_ano

,Estado,Ano,Sorotipo,Casos
0,Acre,2007-06,2,1
1,Acre,2007-06,3,1
2,Acre,2007-06,4,1
3,Acre,2007-06,Sem informação,2355
4,Acre,2009-06,1,10
...,...,...,...,...
1616,Tocantins,2023-06,2,26
1617,Tocantins,2023-06,Sem informação,3222
1618,Tocantins,2024-06,1,37
1619,Tocantins,2024-06,2,58


In [64]:
# exportar dfs
df_sgtf_uf_agrupado_ano.to_csv('df_dengue_pcr_estado_ano.csv', index=False)
df_sgtf_uf_agrupado.to_csv('df_dengue_pcr_estado_periodo.csv', index=False)
df_sgtf_uf_agrupado

,Estado,Período,Sorotipo,Casos,Ano
0,Acre,2007-01,Sem informação,268,2007-06
1,Distrito Federal,2007-01,Sem informação,166,2007-06
2,Distrito Federal,2007-01,1,4,2007-06
3,Rio Grande do Sul,2007-01,Sem informação,35,2007-06
4,Rio de Janeiro,2007-01,3,16,2007-06
...,...,...,...,...,...
11228,Bahia,2024-04,Sem informação,7444,2024-06
11229,Roraima,2024-04,Sem informação,31,2024-06
11230,Mato Grosso do Sul,2024-04,Sem informação,829,2024-06
11231,Mato Grosso,2024-04,1,1,2024-06
